blur detection adapted from: https://pyimagesearch.com/2015/09/07/blur-detection-with-opencv/

In [7]:
from imutils import paths
import argparse
import cv2
import pathlib

In [4]:
def variance_of_laplacian(image):
	# compute the Laplacian of the image and then return the focus
	# measure, which is simply the variance of the Laplacian
	return cv2.Laplacian(image, cv2.CV_64F).var()

In [13]:
cropped_dir = pathlib.Path('../data/processed/cropped')
blur_threshold = 100

In [14]:
blurred_images = []

In [22]:
def check_blurred_images(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow('image', gray)
    fm = variance_of_laplacian(gray)
    if fm < blur_threshold:
        return image_path

In [20]:
for cat_dir in cropped_dir.iterdir():
    if cat_dir.is_dir():
        for image_path in cat_dir.iterdir():
            if image_path.is_file():
                print(check_blurred_images(str(image_path)))

../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_01300004_20190130062959.JPG
../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_01300059_20190130063506.JPG
../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_01300012_20190130063017.JPG
../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_02020544_20190202043913.JPG
../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_01300091_20190130063649.JPG
None
None
None
None
None
None
None
None
None
None
../data/processed/cropped/collared-interact/20190801_kW8J_AucklandIsland_A2_3_41_SD15_20190304_02250287_20190225004026.JPG
None
../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_01300031_20190130063407.JPG
../data/processed/cropped/collared-interact/20190801_kW8J_AucklandIsland_A2_3_41_SD1

error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [23]:
check_blurred_images('../data/processed/cropped/collared-interact/20190801_Y4bG_AucklandIsland_A2_2_38_SD38_20190219_01300004_20190130062959.JPG')

AttributeError: 'numpy.ndarray' object has no attribute 'show'